## Messy Data Cleaning Project

### Import Packages

In [143]:
import csv
import sys
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import os
import numpy as np
import warnings
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer
warnings.filterwarnings("ignore")


## Import Dataset


In [111]:
data = pd.read_csv('Messy_Employee_dataset.csv')

In [112]:
data

,Employee_ID,First_Name,Last_Name,Age,Department_Region,Status,Join_Date,Salary,Email,Phone,Performance_Score,Remote_Work
0,EMP1000,Bob,Davis,25.0,DevOps-California,Active,4/2/2021,59767.65,bob.davis@example.com,-1651623197,Average,True
1,EMP1001,Bob,Brown,NaN,Finance-Texas,Active,7/10/2020,65304.66,bob.brown@example.com,-1898471390,Excellent,True
2,EMP1002,Alice,Jones,NaN,Admin-Nevada,Pending,12/7/2023,88145.90,alice.jones@example.com,-5596363211,Good,True
3,EMP1003,Eva,Davis,25.0,Admin-Nevada,Inactive,11/27/2021,69450.99,eva.davis@example.com,-3476490784,Good,True
4,EMP1004,Frank,Williams,25.0,Cloud Tech-Florida,Active,1/5/2022,109324.61,frank.williams@example.com,-1586734256,Poor,False
...,...,...,...,...,...,...,...,...,...,...,...,...
1015,EMP2015,David,Miller,30.0,HR-California,Active,8/19/2023,NaN,david.miller@example.com,-3546212759,Good,True
1016,EMP2016,David,Johnson,30.0,Cloud Tech-Texas,Inactive,11/7/2021,100215.06,david.johnson@example.com,-2508261122,Good,True
1017,EMP2017,Charlie,Williams,40.0,Finance-New York,Active,10/4/2023,114587.11,charlie.williams@example.com,-1261632487,Average,False
1018,EMP2018,Alice,Garcia,30.0,HR-Florida,Inactive,12/16/2024,71318.79,alice.garcia@example.com,-8995729892,Good,True


In [113]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1020 entries, 0 to 1019
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Employee_ID        1020 non-null   object 
 1   First_Name         1020 non-null   object 
 2   Last_Name          1020 non-null   object 
 3   Age                809 non-null    float64
 4   Department_Region  1020 non-null   object 
 5   Status             1020 non-null   object 
 6   Join_Date          1020 non-null   object 
 7   Salary             996 non-null    float64
 8   Email              1020 non-null   object 
 9   Phone              1020 non-null   int64  
 10  Performance_Score  1020 non-null   object 
 11  Remote_Work        1020 non-null   bool   
dtypes: bool(1), float64(2), int64(1), object(8)
memory usage: 88.8+ KB


In [114]:
data.describe()

,Age,Salary,Phone
count,809.000000,996.000000,1.020000e+03
mean,32.484549,85155.056396,-4.942253e+09
std,5.656860,19873.727918,2.817326e+09
min,25.000000,50047.320000,-9.994973e+09
25%,25.000000,68392.487500,-7.341992e+09
50%,30.000000,85547.870000,-4.943997e+09
75%,40.000000,100974.027500,-2.520391e+09
max,40.000000,119971.650000,-3.896086e+06


## Checking for Missing Values

In [115]:
data.isna().sum()

Employee_ID            0
First_Name             0
Last_Name              0
Age                  211
Department_Region      0
Status                 0
Join_Date              0
Salary                24
Email                  0
Phone                  0
Performance_Score      0
Remote_Work            0
dtype: int64

In [116]:
data['Age'].unique()

array([25., nan, 40., 30., 35.])

## Duplicate Rows

In [117]:
data.duplicated().sum()

0

## Cleaning


In [118]:
data['Department'] = data['Department_Region'].str.split('-').str[0]
data['Region'] = data['Department_Region'].str.split('-').str[1]
data.drop('Department_Region', axis=1, inplace=True)

In [119]:
data[['Department', 'Region']]

,Department,Region
0,DevOps,California
1,Finance,Texas
2,Admin,Nevada
3,Admin,Nevada
4,Cloud Tech,Florida
...,...,...
1015,HR,California
1016,Cloud Tech,Texas
1017,Finance,New York
1018,HR,Florida


In [120]:
data['Phone'] = data['Phone'].astype(str).str.replace('-', '', regex=False)

In [121]:
data['Phone']

0       1651623197
1       1898471390
2       5596363211
3       3476490784
4       1586734256
           ...    
1015    3546212759
1016    2508261122
1017    1261632487
1018    8995729892
1019    7629745492
Name: Phone, Length: 1020, dtype: object

In [122]:
data['Join_Date'] = pd.to_datetime(data['Join_Date'], errors='coerce')

In [123]:
data['Remote_Work'] = data['Remote_Work'].astype(int)
data['Remote_Work']

0       1
1       1
2       1
3       1
4       0
       ..
1015    1
1016    1
1017    0
1018    1
1019    0
Name: Remote_Work, Length: 1020, dtype: int32

In [124]:
data['Performance_Score']

0         Average
1       Excellent
2            Good
3            Good
4            Poor
          ...    
1015         Good
1016         Good
1017      Average
1018         Good
1019      Average
Name: Performance_Score, Length: 1020, dtype: object

In [125]:
performance_map = {'Excellent': 4, 'Good': 3, 'Average': 2, 'Poor': 1}
data['Performance_Score'] = data['Performance_Score'].map(performance_map)
data['Performance_Score']

0       2
1       4
2       3
3       3
4       1
       ..
1015    3
1016    3
1017    2
1018    3
1019    2
Name: Performance_Score, Length: 1020, dtype: int64

In [126]:
data['Status'].unique()

array(['Active', 'Pending', 'Inactive'], dtype=object)

In [127]:
OHC = OneHotEncoder(sparse_output=False, handle_unknown='ignore')


In [128]:
data['Status'] = OHC.fit_transform(data[['Status']]).astype(int)
data['Status']

0       1
1       1
2       0
3       0
4       1
       ..
1015    1
1016    0
1017    1
1018    0
1019    0
Name: Status, Length: 1020, dtype: int32

In [129]:
data

,Employee_ID,First_Name,Last_Name,Age,Status,Join_Date,Salary,Email,Phone,Performance_Score,Remote_Work,Department,Region
0,EMP1000,Bob,Davis,25.0,1,2021-04-02,59767.65,bob.davis@example.com,1651623197,2,1,DevOps,California
1,EMP1001,Bob,Brown,NaN,1,2020-07-10,65304.66,bob.brown@example.com,1898471390,4,1,Finance,Texas
2,EMP1002,Alice,Jones,NaN,0,2023-12-07,88145.90,alice.jones@example.com,5596363211,3,1,Admin,Nevada
3,EMP1003,Eva,Davis,25.0,0,2021-11-27,69450.99,eva.davis@example.com,3476490784,3,1,Admin,Nevada
4,EMP1004,Frank,Williams,25.0,1,2022-01-05,109324.61,frank.williams@example.com,1586734256,1,0,Cloud Tech,Florida
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1015,EMP2015,David,Miller,30.0,1,2023-08-19,NaN,david.miller@example.com,3546212759,3,1,HR,California
1016,EMP2016,David,Johnson,30.0,0,2021-11-07,100215.06,david.johnson@example.com,2508261122,3,1,Cloud Tech,Texas
1017,EMP2017,Charlie,Williams,40.0,1,2023-10-04,114587.11,charlie.williams@example.com,1261632487,2,0,Finance,New York
1018,EMP2018,Alice,Garcia,30.0,0,2024-12-16,71318.79,alice.garcia@example.com,8995729892,3,1,HR,Florida


## KNN Imputing
For the variable columns Age and Salary, after some research of a better practice to utilise KNN Imputation rather than the easier option of replacing those missing values with the average/median, because doing so would result in the data losing its statistical variance and structure and its real world relation in the data. 

In [130]:
data['Join_Year'] = data['Join_Date'].dt.year
data['Join_Month'] = data['Join_Date'].dt.month
data['Join_Day'] = data['Join_Date'].dt.day

In [131]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1020 entries, 0 to 1019
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Employee_ID        1020 non-null   object        
 1   First_Name         1020 non-null   object        
 2   Last_Name          1020 non-null   object        
 3   Age                809 non-null    float64       
 4   Status             1020 non-null   int32         
 5   Join_Date          1020 non-null   datetime64[ns]
 6   Salary             996 non-null    float64       
 7   Email              1020 non-null   object        
 8   Phone              1020 non-null   object        
 9   Performance_Score  1020 non-null   int64         
 10  Remote_Work        1020 non-null   int32         
 11  Department         1020 non-null   object        
 12  Region             1020 non-null   object        
 13  Join_Year          1020 non-null   int32         
 14  Join_Mon

In [132]:
X = data.drop(columns=['Employee_ID', 'First_Name', 'Last_Name', 'Join_Date', 'Phone', 'Email'])

In [133]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1020 entries, 0 to 1019
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Age                809 non-null    float64
 1   Status             1020 non-null   int32  
 2   Salary             996 non-null    float64
 3   Performance_Score  1020 non-null   int64  
 4   Remote_Work        1020 non-null   int32  
 5   Department         1020 non-null   object 
 6   Region             1020 non-null   object 
 7   Join_Year          1020 non-null   int32  
 8   Join_Month         1020 non-null   int32  
 9   Join_Day           1020 non-null   int32  
dtypes: float64(2), int32(5), int64(1), object(2)
memory usage: 59.9+ KB


In [134]:
X_dept_ohe = OHC.fit_transform(X[['Department', 'Region']])

In [135]:
X_dept_ohe_cols = OHC.get_feature_names_out(['Department', 'Region'])

In [136]:
X_dept = pd.DataFrame(X_dept_ohe, columns=X_dept_ohe_cols)
X_dept

,Department_Admin,Department_Cloud Tech,Department_DevOps,Department_Finance,Department_HR,Department_Sales,Region_California,Region_Florida,Region_Illinois,Region_Nevada,Region_New York,Region_Texas
0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1015,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1016,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1017,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1018,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [137]:
X = X.drop(columns=['Department', 'Region'])
X = pd.concat([X, X_dept], axis=1)

In [138]:
X

,Age,Status,Salary,Performance_Score,Remote_Work,Join_Year,Join_Month,Join_Day,Department_Admin,Department_Cloud Tech,Department_DevOps,Department_Finance,Department_HR,Department_Sales,Region_California,Region_Florida,Region_Illinois,Region_Nevada,Region_New York,Region_Texas
0,25.0,1,59767.65,2,1,2021,4,2,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,NaN,1,65304.66,4,1,2020,7,10,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,NaN,0,88145.90,3,1,2023,12,7,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,25.0,0,69450.99,3,1,2021,11,27,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,25.0,1,109324.61,1,0,2022,1,5,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1015,30.0,1,NaN,3,1,2023,8,19,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1016,30.0,0,100215.06,3,1,2021,11,7,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1017,40.0,1,114587.11,2,0,2023,10,4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1018,30.0,0,71318.79,3,1,2024,12,16,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [139]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1020 entries, 0 to 1019
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Age                    809 non-null    float64
 1   Status                 1020 non-null   int32  
 2   Salary                 996 non-null    float64
 3   Performance_Score      1020 non-null   int64  
 4   Remote_Work            1020 non-null   int32  
 5   Join_Year              1020 non-null   int32  
 6   Join_Month             1020 non-null   int32  
 7   Join_Day               1020 non-null   int32  
 8   Department_Admin       1020 non-null   float64
 9   Department_Cloud Tech  1020 non-null   float64
 10  Department_DevOps      1020 non-null   float64
 11  Department_Finance     1020 non-null   float64
 12  Department_HR          1020 non-null   float64
 13  Department_Sales       1020 non-null   float64
 14  Region_California      1020 non-null   float64
 15  Regi

In [144]:
scaler = StandardScaler()
x_scaled = scaler.fit_transform(X)
KNN = KNNImputer(n_neighbors=5, weights="distance")

In [145]:
knn_arr = KNN.fit_transform(x_scaled)
knn_arr = scaler.inverse_transform(knn_arr)
imputed = pd.DataFrame(knn_arr, columns=X.columns)
imputed

,Age,Status,Salary,Performance_Score,Remote_Work,Join_Year,Join_Month,Join_Day,Department_Admin,Department_Cloud Tech,Department_DevOps,Department_Finance,Department_HR,Department_Sales,Region_California,Region_Florida,Region_Illinois,Region_Nevada,Region_New York,Region_Texas
0,25.000000,1.0,59767.650000,2.0,1.0,2021.0,4.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,31.973372,1.0,65304.660000,4.0,1.0,2020.0,7.0,10.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,34.932868,0.0,88145.900000,3.0,1.0,2023.0,12.0,7.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,25.000000,0.0,69450.990000,3.0,1.0,2021.0,11.0,27.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,25.000000,1.0,109324.610000,1.0,0.0,2022.0,1.0,5.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1015,30.000000,1.0,87173.329846,3.0,1.0,2023.0,8.0,19.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1016,30.000000,0.0,100215.060000,3.0,1.0,2021.0,11.0,7.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1017,40.000000,1.0,114587.110000,2.0,0.0,2023.0,10.0,4.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1018,30.000000,0.0,71318.790000,3.0,1.0,2024.0,12.0,16.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [146]:
imputed['Age'] = (imputed['Age']/5).round() * 5

In [148]:
imputed

,Age,Status,Salary,Performance_Score,Remote_Work,Join_Year,Join_Month,Join_Day,Department_Admin,Department_Cloud Tech,Department_DevOps,Department_Finance,Department_HR,Department_Sales,Region_California,Region_Florida,Region_Illinois,Region_Nevada,Region_New York,Region_Texas
0,25.0,1.0,59767.650000,2.0,1.0,2021.0,4.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,30.0,1.0,65304.660000,4.0,1.0,2020.0,7.0,10.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,35.0,0.0,88145.900000,3.0,1.0,2023.0,12.0,7.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,25.0,0.0,69450.990000,3.0,1.0,2021.0,11.0,27.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,25.0,1.0,109324.610000,1.0,0.0,2022.0,1.0,5.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1015,30.0,1.0,87173.329846,3.0,1.0,2023.0,8.0,19.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1016,30.0,0.0,100215.060000,3.0,1.0,2021.0,11.0,7.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1017,40.0,1.0,114587.110000,2.0,0.0,2023.0,10.0,4.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1018,30.0,0.0,71318.790000,3.0,1.0,2024.0,12.0,16.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [149]:
data.drop(columns=['Join_Year'], inplace=True)

In [151]:
data.drop(columns=['Join_Month','Join_Day'], inplace=True)

In [152]:
data['Age'] = imputed['Age']
data['Salary'] = imputed['Salary']
data.isnull().sum()

Employee_ID          0
First_Name           0
Last_Name            0
Age                  0
Status               0
Join_Date            0
Salary               0
Email                0
Phone                0
Performance_Score    0
Remote_Work          0
Department           0
Region               0
dtype: int64

In [153]:
reorder = ['Employee_ID', 'First_Name', 'Last_Name',
       'Department', 'Region' ,'Age', 'Status', 'Join_Date',
       'Salary', 'Email', 'Phone', 'Performance_Score', 'Remote_Work']

In [154]:
data = data[reorder]

In [155]:
data

,Employee_ID,First_Name,Last_Name,Department,Region,Age,Status,Join_Date,Salary,Email,Phone,Performance_Score,Remote_Work
0,EMP1000,Bob,Davis,DevOps,California,25.0,1,2021-04-02,59767.650000,bob.davis@example.com,1651623197,2,1
1,EMP1001,Bob,Brown,Finance,Texas,30.0,1,2020-07-10,65304.660000,bob.brown@example.com,1898471390,4,1
2,EMP1002,Alice,Jones,Admin,Nevada,35.0,0,2023-12-07,88145.900000,alice.jones@example.com,5596363211,3,1
3,EMP1003,Eva,Davis,Admin,Nevada,25.0,0,2021-11-27,69450.990000,eva.davis@example.com,3476490784,3,1
4,EMP1004,Frank,Williams,Cloud Tech,Florida,25.0,1,2022-01-05,109324.610000,frank.williams@example.com,1586734256,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1015,EMP2015,David,Miller,HR,California,30.0,1,2023-08-19,87173.329846,david.miller@example.com,3546212759,3,1
1016,EMP2016,David,Johnson,Cloud Tech,Texas,30.0,0,2021-11-07,100215.060000,david.johnson@example.com,2508261122,3,1
1017,EMP2017,Charlie,Williams,Finance,New York,40.0,1,2023-10-04,114587.110000,charlie.williams@example.com,1261632487,2,0
1018,EMP2018,Alice,Garcia,HR,Florida,30.0,0,2024-12-16,71318.790000,alice.garcia@example.com,8995729892,3,1


In [156]:
for i in data.columns:
    print(pd.DataFrame(data[i].unique()))

            0
0     EMP1000
1     EMP1001
2     EMP1002
3     EMP1003
4     EMP1004
...       ...
1015  EMP2015
1016  EMP2016
1017  EMP2017
1018  EMP2018
1019  EMP2019

[1020 rows x 1 columns]
         0
0      Bob
1    Alice
2      Eva
3    Frank
4  Charlie
5    David
6    Heidi
7    Grace
          0
0     Davis
1     Brown
2     Jones
3  Williams
4    Garcia
5   Johnson
6    Miller
7     Smith
            0
0      DevOps
1     Finance
2       Admin
3  Cloud Tech
4       Sales
5          HR
            0
0  California
1       Texas
2      Nevada
3     Florida
4    New York
5    Illinois
      0
0  25.0
1  30.0
2  35.0
3  40.0
   0
0  1
1  0
             0
0   2021-04-02
1   2020-07-10
2   2023-12-07
3   2021-11-27
4   2022-01-05
..         ...
755 2021-07-02
756 2022-12-05
757 2024-09-24
758 2023-08-16
759 2021-01-28

[760 rows x 1 columns]
                  0
0      59767.650000
1      65304.660000
2      88145.900000
3      69450.990000
4     109324.610000
...             ...
997  

In [157]:
data.to_csv('Cleaned_Employee_dataset.csv', index=False)